In [1]:
import os
import pandas as pd
import seaborn as sns
import numpy as np

import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
#File path to the csv file
csv_file = "./.data/ag_news_csv/train.csv"

# Read csv file into dataframe
df = pd.read_csv(csv_file, names=["label", "title", "text"])

# Print first 5 rows in the dataframe
df.head()

,label,title,text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [3]:
df['label'].value_counts()

4    30000
3    30000
2    30000
1    30000
Name: label, dtype: int64

In [4]:
label_file = "./.data/ag_news_csv/classes.txt"

In [5]:
with open(label_file) as f:
    file_contents = f.read()
    print(file_contents)

World
Sports
Business
Sci/Tech



In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
text = df.text.values
wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Split Data to Test and Train

In [ ]:
X,y = df.text, df.label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Visualize how text turns into numbers ([word embedding](https://en.wikipedia.org/wiki/Word_embedding))

In [ ]:
def get_vector_feature_matrix():
    vectorizer = CountVectorizer(lowercase=True, stop_words="english", max_features=5)
    #vectorizer = CountVectorizer(lowercase=True, stop_words="english", max_features=4000)
    #vectorizer = CountVectorizer(lowercase=True, stop_words="english",ngram_range=(1, 2), max_features=5000)
    #vectorizer = CountVectorizer(lowercase=True, stop_words="english", tokenizer=stemming_tokenizer) 
    vector = vectorizer.fit_transform(np.array(df.text))
    return vector, vectorizer

In [ ]:
vector, vectorizer = get_vector_feature_matrix()

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
print(vector.toarray()[500])

In [ ]:
df['text'].iloc[500]

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
x = vectorizer.transform(np.array([df['text'].iloc[500]]))
print(x)

In [ ]:
print(len(vectorizer.vocabulary_))

## Create and Test Model

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words="english",ngram_range=(1, 2), max_features=100000)),
    ('clf', LogisticRegression(multi_class='ovr',max_iter=500)),
])

pipeline.fit(X_train, y_train)

In [ ]:
accuracy = pipeline.score(X_test, y_test)
print ("Accuracy is {}".format(accuracy))

In [ ]:
x_text = ["MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."]

In [ ]:
x = np.array(x_text)
proba = pipeline.predict_proba(x_text)
classes = pipeline.classes_
resultdf = pd.DataFrame(data=proba, columns=classes)

In [ ]:
result =pipeline.predict(x)[0]

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

print(f"Predicted: {result}")
print("This is a %s news article" %ag_news_label[result])

In [ ]:
resultdf